<a href="https://colab.research.google.com/github/Alenushka2013/ML_for_people_lectures/blob/main/Lecture_2_4_%D0%9F%D0%B5%D1%80%D0%B5%D1%85%D1%80%D0%B5%D1%81%D0%BD%D0%B0_%D0%B2%D0%B0%D0%BB%D1%96%D0%B4%D0%B0%D1%86%D1%96%D1%8F_%D0%9F%D0%BE%D1%88%D1%83%D0%BA_%D0%BE%D0%BF%D1%82%D0%B8%D0%BC%D0%B0%D0%BB%D1%8C%D0%BD%D0%B8%D1%85_%D0%B3%D1%96%D0%BF%D0%B5%D1%80%D0%BF%D0%B0%D1%80%D0%B0%D0%BC%D0%B5%D1%82%D1%80%D1%96%D0%B2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Перехресна валідація (Cross Validation)

## k-кратна перехресна валідація (k-fold Cross Validation)

Ця техніка розбиває дані на k підмножин (folds) і використовує k-1 підмножин для тренування моделі, а одну підмножину для тестування. Процес повторюється k разів, кожного разу використовуючи іншу підмножину для тестування.

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold

X = np.array([[101, 202], [303, 404], [102, 204], [306, 408]])
y = np.array([0, 1, 2, 3])

In [2]:
X

array([[101, 202],
       [303, 404],
       [102, 204],
       [306, 408]])

In [3]:
# ?KFold

In [4]:
kf = KFold(n_splits=3)

In [5]:
kf.split(X)

<generator object _BaseKFold.split at 0x7859b7669fc0>

In [6]:
for id_, (train_index, test_index) in enumerate(kf.split(X)):
    print(f'Experiment {id_}')
    print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    print("X")
    print(X_train, X_test, end='\n')
    print("y")
    print(y_train, y_test, end='\n\n')

Experiment 0
TRAIN: [2 3] TEST: [0 1]
X
[[102 204]
 [306 408]] [[101 202]
 [303 404]]
y
[2 3] [0 1]

Experiment 1
TRAIN: [0 1 3] TEST: [2]
X
[[101 202]
 [303 404]
 [306 408]] [[102 204]]
y
[0 1 3] [2]

Experiment 2
TRAIN: [0 1 2] TEST: [3]
X
[[101 202]
 [303 404]
 [102 204]] [[306 408]]
y
[0 1 2] [3]



## Leave-one-out техніка
Варіант кросвалідації, коли кількість розбиттів дорівнює кількості екземплярів в даних.

In [7]:
from sklearn.model_selection import LeaveOneOut

X = np.array([[101, 202], [303, 404], [102, 204], [306, 408], [404, 202]])
y = np.array([1, 2, 3, 4, 5])
loo = LeaveOneOut()

for train_index, test_index in loo.split(X):
    print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

TRAIN: [1 2 3 4] TEST: [0]
TRAIN: [0 2 3 4] TEST: [1]
TRAIN: [0 1 3 4] TEST: [2]
TRAIN: [0 1 2 4] TEST: [3]
TRAIN: [0 1 2 3] TEST: [4]


## Стратифікована k-кратна (Stratified k-fold)

Ця техніка крос-валідації використовується для збереження пропорції класів у кожній складці (fold), що особливо важливо для незбалансованих наборів даних.

In [8]:
from sklearn.model_selection import StratifiedKFold

X = np.array([[101, 202], [102, 204], [303, 404], [306, 408], [101, 202], [505, 102]])
y = np.array([0, 0, 1, 1, 0, 1])
skf = StratifiedKFold(n_splits=3)

for train_index, test_index in skf.split(X, y):
    print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

TRAIN: [1 3 4 5] TEST: [0 2]
TRAIN: [0 2 4 5] TEST: [1 3]
TRAIN: [0 1 2 3] TEST: [4 5]


# Пошук оптимальних гіперпараметрів

Завантажимо набір даних.

In [9]:
from sklearn.datasets import fetch_california_housing
from sklearn.metrics import mean_squared_error, root_mean_squared_error
from sklearn.model_selection import train_test_split


def get_indices(X):
    train_full_id, test_id = train_test_split(X.index, test_size=test_size, shuffle=True, random_state=42)
    train_id, val_id = train_test_split(train_full_id, test_size=test_size, shuffle=True, random_state=42)
    return train_id, val_id, test_id

def get_metrics(y_true, y_predicted, n_digits=5):
    params = dict(y_true=y_true, y_pred=y_predicted)
    mse = mean_squared_error(**params)
    rmse = root_mean_squared_error(**params)
    return dict(mse=round(mse, n_digits), rmse=round(rmse, n_digits))

In [10]:
dataset = fetch_california_housing()

In [11]:
dataset.keys()

dict_keys(['data', 'target', 'frame', 'target_names', 'feature_names', 'DESCR'])

In [12]:
dataset = fetch_california_housing() # завантаження датасету і всієї інформації у вигляді словнику
df = pd.DataFrame(dataset['data'], columns = dataset['feature_names']) # вибір даних і назв стовбчиків
target_col_name = dataset['target_names'][0] # цільова змінна
df[target_col_name] = dataset['target'] # колонка з цільовими значеннями
del dataset

In [13]:
df.head()

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,MedHouseVal
0,8.3252,41.0,6.984127,1.023810,322.0,2.555556,37.88,-122.23,4.526
1,8.3014,21.0,6.238137,0.971880,2401.0,2.109842,37.86,-122.22,3.585
2,7.2574,52.0,8.288136,1.073446,496.0,2.802260,37.85,-122.24,3.521
3,5.6431,52.0,5.817352,1.073059,558.0,2.547945,37.85,-122.25,3.413
4,3.8462,52.0,6.281853,1.081081,565.0,2.181467,37.85,-122.25,3.422


In [14]:
X = df.drop([target_col_name],axis=1)
y = df[target_col_name]

In [15]:
test_size = 0.2

train_id, val_id, test_id = get_indices(X)


In [16]:
len(train_id), len(val_id), len(test_id)

(13209, 3303, 4128)

## Пошук по сітці (Grid search)

Пошук по сітці (Grid search) - це метод для налаштування гіперпараметрів моделі машинного навчання. Він передбачає перебір усіх можливих комбінацій заданих гіперпараметрів для визначення найкращої комбінації, яка забезпечує найвищу продуктивність моделі.

In [17]:
from sklearn.linear_model import Lasso, ElasticNet
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.pipeline import Pipeline

In [18]:
degree = 2
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('poly_features', PolynomialFeatures(degree=degree)),
    ('model', Lasso())
])

In [19]:
np.arange(0,10,0.1).shape

(100,)

In [20]:
search = GridSearchCV(
    pipeline,
    {'model__alpha':np.arange(0,10,0.1)}, # гіперпараметр для моделі Lasso()
    cv = 5, # фолди
    scoring="neg_mean_squared_error", # помилка
    verbose=3  # рівень відображеня інформації 3 найбільше записів
)

In [21]:
search.fit(X.loc[train_id], y.loc[train_id])

Fitting 5 folds for each of 100 candidates, totalling 500 fits


/usr/local/lib/python3.11/dist-packages/sklearn/base.py:1389: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimator
  return fit_method(estimator, *args, **kwargs)
/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_coordinate_descent.py:695: UserWarning: Coordinate descent with no regularization may lead to unexpected results and is discouraged.
  model = cd_fast.enet_coordinate_descent(
/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.155e+03, tolerance: 1.397e+00 Linear regression models with null weight for the l1 regularization term are more efficiently fitted using one of the solvers implemented in sklearn.linear_model.Ridge/RidgeCV instead.
  model = cd_fast.enet_coordinate_descent

[CV 1/5] END .................model__alpha=0.0;, score=-3.869 total time=   2.9s


/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.181e+03, tolerance: 1.395e+00 Linear regression models with null weight for the l1 regularization term are more efficiently fitted using one of the solvers implemented in sklearn.linear_model.Ridge/RidgeCV instead.
  model = cd_fast.enet_coordinate_descent(
/usr/local/lib/python3.11/dist-packages/sklearn/base.py:1389: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimator
  return fit_method(estimator, *args, **kwargs)
/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_coordinate_descent.py:695: UserWarning: Coordinate descent with no regularization may lead to unexpected results and is discouraged.
  model = cd_fast.enet_coordinate_descent

[CV 2/5] END ................model__alpha=0.0;, score=-56.554 total time=   1.5s


...

/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.185e+03, tolerance: 1.391e+00 Linear regression models with null weight for the l1 regularization term are more efficiently fitted using one of the solvers implemented in sklearn.linear_model.Ridge/RidgeCV instead.
  model = cd_fast.enet_coordinate_descent(
/usr/local/lib/python3.11/dist-packages/sklearn/base.py:1389: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimator
  return fit_method(estimator, *args, **kwargs)
/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_coordinate_descent.py:695: UserWarning: Coordinate descent with no regularization may lead to unexpected results and is discouraged.
  model = cd_fast.enet_coordinate_descent

[CV 4/5] END .................model__alpha=0.0;, score=-0.443 total time=   2.5s


/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.213e+03, tolerance: 1.415e+00 Linear regression models with null weight for the l1 regularization term are more efficiently fitted using one of the solvers implemented in sklearn.linear_model.Ridge/RidgeCV instead.
  model = cd_fast.enet_coordinate_descent(


[CV 5/5] END .................model__alpha=0.0;, score=-0.417 total time=   4.9s
[CV 1/5] END .................model__alpha=0.1;, score=-0.671 total time=   0.0s
[CV 2/5] END .................model__alpha=0.1;, score=-0.663 total time=   0.0s
[CV 3/5] END .................model__alpha=0.1;, score=-0.652 total time=   0.0s
[CV 4/5] END .................model__alpha=0.1;, score=-0.678 total time=   0.0s
[CV 5/5] END .................model__alpha=0.1;, score=-0.657 total time=   0.0s
[CV 1/5] END .................model__alpha=0.2;, score=-0.741 total time=   0.0s
[CV 2/5] END .................model__alpha=0.2;, score=-0.737 total time=   0.0s
[CV 3/5] END .................model__alpha=0.2;, score=-0.719 total time=   0.0s
[CV 4/5] END .................model__alpha=0.2;, score=-0.756 total time=   0.0s
[CV 5/5] END .................model__alpha=0.2;, score=-0.715 total time=   0.0s
[CV 1/5] END .model__alpha=0.30000000000000004;, score=-0.795 total time=   0.0s
[CV 2/5] END .model__alpha=0

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('scaler', StandardScaler()),
                                       ('poly_features', PolynomialFeatures()),
                                       ('model', Lasso())]),
             param_grid={'model__alpha': array([0. , 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1. , 1.1, 1.2,
       1.3, 1.4, 1.5, 1.6, 1.7, 1.8, 1.9, 2. , 2.1, 2.2, 2.3, 2.4, 2.5,
       2.6, 2.7, 2.8, 2.9, 3. , 3.1, 3.2, 3.3, 3.4, 3.5, 3.6, 3.7, 3.8,
       3.9, 4. , 4.1, 4.2, 4.3, 4.4, 4.5, 4.6, 4.7, 4.8, 4.9, 5. , 5.1,
       5.2, 5.3, 5.4, 5.5, 5.6, 5.7, 5.8, 5.9, 6. , 6.1, 6.2, 6.3, 6.4,
       6.5, 6.6, 6.7, 6.8, 6.9, 7. , 7.1, 7.2, 7.3, 7.4, 7.5, 7.6, 7.7,
       7.8, 7.9, 8. , 8.1, 8.2, 8.3, 8.4, 8.5, 8.6, 8.7, 8.8, 8.9, 9. ,
       9.1, 9.2, 9.3, 9.4, 9.5, 9.6, 9.7, 9.8, 9.9])},
             scoring='neg_mean_squared_error', verbose=3)

In [22]:
search.best_params_  # натренована армія модлелей - кращий параметр для нашої моделі при обраній метриці

{'model__alpha': np.float64(0.1)}

In [23]:
search.best_estimator_ # вертає pipeline з конктретими значенями гіперпараметрів

Pipeline(steps=[('scaler', StandardScaler()),
                ('poly_features', PolynomialFeatures()),
                ('model', Lasso(alpha=np.float64(0.1)))])

In [24]:
search.best_estimator_.predict(X.loc[val_id])

array([2.01108779, 1.26290534, 2.07831121, ..., 1.90289166, 1.93371759,
       1.91129538])

In [25]:
coefficients = search.best_estimator_.named_steps['model'].coef_  # модна подивитися коефіцієнти моделі

Видимо, серед коефіцієнтів у нас вийшло багато нулів і фактично значущими залишилися лише декілька.

In [26]:
coefficients.round(2)

array([ 0.  ,  0.71,  0.1 , -0.  , -0.  , -0.  , -0.  , -0.  , -0.  ,
       -0.  ,  0.  ,  0.  , -0.  , -0.  , -0.  ,  0.  , -0.  ,  0.  ,
        0.  ,  0.  ,  0.  , -0.  ,  0.  , -0.  ,  0.  , -0.  ,  0.  ,
        0.  , -0.  , -0.  , -0.  ,  0.  ,  0.  , -0.  , -0.  , -0.  ,
        0.  ,  0.  , -0.  , -0.  , -0.  , -0.  , -0.02,  0.  ,  0.  ])

Давайте дізнаємось, які коефіцієнти є найбільш значущими:

In [27]:
# dir(search.best_estimator_.named_steps['poly_features'])

In [28]:
feature_names = search.best_estimator_.named_steps['poly_features'].get_feature_names_out()
coef_df = pd.DataFrame(zip(feature_names, coefficients.round(3)), columns=['feature_name', 'coef'])

In [29]:
coef_df.sort_values('coef', ascending=False)[:10]

,feature_name,coef
1,x0,0.706
2,x1,0.105
0,1,0.000
3,x2,-0.000
4,x3,-0.000
5,x4,-0.000
6,x5,-0.000
8,x7,-0.000
9,x0^2,-0.000
11,x0 x2,0.000


Словник з назвами ознак:

In [30]:
X.columns.to_frame().reset_index(drop=True).to_dict()[0]

{0: 'MedInc',
 1: 'HouseAge',
 2: 'AveRooms',
 3: 'AveBedrms',
 4: 'Population',
 5: 'AveOccup',
 6: 'Latitude',
 7: 'Longitude'}

Видно, що все одно найважливіша ознака - MedInc без будь-яких ступенів.

In [31]:
search.best_score_

np.float64(-0.6640858768504454)

In [32]:
best_model = search.best_estimator_

y_train_pred = best_model.predict(X.loc[train_id])
y_val_pred = best_model.predict(X.loc[val_id])
train_metrics = get_metrics(y.loc[train_id], y_train_pred)
val_metrics = get_metrics(y.loc[val_id], y_val_pred)

In [33]:
train_metrics, val_metrics

({'mse': 0.66241, 'rmse': 0.81389}, {'mse': 0.69114, 'rmse': 0.83135})

## Випадковий пошук (Random Search)

Випадковий пошук - це метод гіперпараметричної оптимізації, який випадковим чином вибирає комбінації гіперпараметрів з визначеного простору.

In [34]:
from sklearn.model_selection import RandomizedSearchCV

In [35]:
params = dict()

# значения для alpha: 100 значений мужду e^-5 и e^5
params['alpha'] =  np.logspace(-5, 5, 100, endpoint=True)

# значения для l1_ratio: 100 значений между 0 и 1
params['l1_ratio'] = np.arange(0, 1, 0.01)

In [36]:
len(params['alpha'])*len(params['l1_ratio'])

10000

In [37]:
model = ElasticNet()

In [38]:
#?RandomizedSearchCV

Які стандартні методи оцінки якості моделі доступні в `RandomizedSearchCV` та `GridSearchCV`: https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter

In [39]:
search = RandomizedSearchCV(
    model,
    params,
    n_iter = 100,
    cv=5,
    scoring="neg_mean_squared_error",
    verbose=3,
    refit=True
)

In [40]:
search.fit(X.loc[train_id], y.loc[train_id])

Fitting 5 folds for each of 100 candidates, totalling 500 fits
[CV 1/5] END alpha=2.0092330025650458e-05, l1_ratio=0.93;, score=-0.527 total time=   0.0s
[CV 2/5] END alpha=2.0092330025650458e-05, l1_ratio=0.93;, score=-0.508 total time=   0.0s
[CV 3/5] END alpha=2.0092330025650458e-05, l1_ratio=0.93;, score=-0.499 total time=   0.0s
[CV 4/5] END alpha=2.0092330025650458e-05, l1_ratio=0.93;, score=-0.529 total time=   0.0s
[CV 5/5] END alpha=2.0092330025650458e-05, l1_ratio=0.93;, score=-0.518 total time=   0.0s


/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 6.974e+03, tolerance: 1.397e+00 Linear regression models with null weight for the l1 regularization term are more efficiently fitted using one of the solvers implemented in sklearn.linear_model.Ridge/RidgeCV instead.
  model = cd_fast.enet_coordinate_descent(
/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 6.967e+03, tolerance: 1.395e+00 Linear regression models with null weight for the l1 regularization term are more efficiently fitted using one of the solvers implemented in sklearn.linear_mode

[CV 1/5] END alpha=7742.636826811277, l1_ratio=0.0;, score=-1.340 total time=   0.1s
[CV 2/5] END alpha=7742.636826811277, l1_ratio=0.0;, score=-1.345 total time=   0.1s


/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 7.029e+03, tolerance: 1.407e+00 Linear regression models with null weight for the l1 regularization term are more efficiently fitted using one of the solvers implemented in sklearn.linear_model.Ridge/RidgeCV instead.
  model = cd_fast.enet_coordinate_descent(
/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 6.944e+03, tolerance: 1.391e+00 Linear regression models with null weight for the l1 regularization term are more efficiently fitted using one of the solvers implemented in sklearn.linear_mode

[CV 3/5] END alpha=7742.636826811277, l1_ratio=0.0;, score=-1.299 total time=   0.1s
[CV 4/5] END alpha=7742.636826811277, l1_ratio=0.0;, score=-1.363 total time=   0.1s


/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 7.064e+03, tolerance: 1.415e+00 Linear regression models with null weight for the l1 regularization term are more efficiently fitted using one of the solvers implemented in sklearn.linear_model.Ridge/RidgeCV instead.
  model = cd_fast.enet_coordinate_descent(


[CV 5/5] END alpha=7742.636826811277, l1_ratio=0.0;, score=-1.273 total time=   0.1s
[CV 1/5] END alpha=31257.15849688235, l1_ratio=0.22;, score=-1.340 total time=   0.0s
[CV 2/5] END alpha=31257.15849688235, l1_ratio=0.22;, score=-1.348 total time=   0.0s
[CV 3/5] END alpha=31257.15849688235, l1_ratio=0.22;, score=-1.303 total time=   0.0s
[CV 4/5] END alpha=31257.15849688235, l1_ratio=0.22;, score=-1.364 total time=   0.0s
[CV 5/5] END alpha=31257.15849688235, l1_ratio=0.22;, score=-1.275 total time=   0.0s
[CV 1/5] END alpha=9.111627561154886, l1_ratio=0.53;, score=-1.341 total time=   0.0s
[CV 2/5] END alpha=9.111627561154886, l1_ratio=0.53;, score=-1.347 total time=   0.0s
[CV 3/5] END alpha=9.111627561154886, l1_ratio=0.53;, score=-1.300 total time=   0.0s
[CV 4/5] END alpha=9.111627561154886, l1_ratio=0.53;, score=-1.364 total time=   0.0s
[CV 5/5] END alpha=9.111627561154886, l1_ratio=0.53;, score=-1.274 total time=   0.0s
[CV 1/5] END alpha=1519.9110829529332, l1_ratio=0.59;, 

RandomizedSearchCV(cv=5, estimator=ElasticNet(), n_iter=100,
                   param_distributions={'alpha': array([1.00000000e-05, 1.26185688e-05, 1.59228279e-05, 2.00923300e-05,
       2.53536449e-05, 3.19926714e-05, 4.03701726e-05, 5.09413801e-05,
       6.42807312e-05, 8.11130831e-05, 1.02353102e-04, 1.29154967e-04,
       1.62975083e-04, 2.05651231e-04, 2.59502421e-04, 3.27454916e-04,
       4.13201240e-04, 5...
       0.33, 0.34, 0.35, 0.36, 0.37, 0.38, 0.39, 0.4 , 0.41, 0.42, 0.43,
       0.44, 0.45, 0.46, 0.47, 0.48, 0.49, 0.5 , 0.51, 0.52, 0.53, 0.54,
       0.55, 0.56, 0.57, 0.58, 0.59, 0.6 , 0.61, 0.62, 0.63, 0.64, 0.65,
       0.66, 0.67, 0.68, 0.69, 0.7 , 0.71, 0.72, 0.73, 0.74, 0.75, 0.76,
       0.77, 0.78, 0.79, 0.8 , 0.81, 0.82, 0.83, 0.84, 0.85, 0.86, 0.87,
       0.88, 0.89, 0.9 , 0.91, 0.92, 0.93, 0.94, 0.95, 0.96, 0.97, 0.98,
       0.99])},
                   scoring='neg_mean_squared_error', verbose=3)

In [41]:
display(search.best_estimator_, search.best_score_)

ElasticNet(alpha=np.float64(0.00210490414451202), l1_ratio=np.float64(0.2))

np.float64(-0.5162196481284268)

In [42]:
ElasticNet()

ElasticNet()

In [43]:
best_model = search.best_estimator_

y_train_pred = best_model.predict(X.loc[train_id])
y_val_pred = best_model.predict(X.loc[val_id])
train_metrics = get_metrics(y.loc[train_id], y_train_pred)
val_metrics = get_metrics(y.loc[val_id], y_val_pred)

In [44]:
train_metrics, val_metrics

({'mse': 0.51302, 'rmse': 0.71626}, {'mse': 0.53883, 'rmse': 0.73405})

Нам вдалось якісно покращити модель. Ура!